In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from surprise import Dataset,Reader,NMF,accuracy
from surprise.model_selection import train_test_split,KFold
from utils import Dataloader

import joblib
from collections import defaultdict

In [2]:
DIR_PATH = "./data/"
users_df = Dataloader.load_users(DIR_PATH)
ratings_df = Dataloader.load_ratings(DIR_PATH)
movies_df = Dataloader.load_movies(DIR_PATH)

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

In [4]:
trainset, testset = train_test_split(data, test_size=0.2)

In [14]:
NMF_model = NMF()

In [15]:
NMF_model.fit(trainset)
NMF_model_predictions = NMF_model.test(testset)
NMF_model_predictions_df = pd.DataFrame(NMF_model_predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])

In [16]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [17]:
rmse = accuracy.rmse(NMF_model_predictions)
mae = accuracy.mae(NMF_model_predictions)

print("RMSE:", rmse)
print("MAE:", mae)

RMSE: 0.9187
MAE:  0.7258
RMSE: 0.9187241307434599
MAE: 0.7257540006519875


In [18]:
NMF_model_predictions_list = NMF_model_predictions_df.values.tolist()

# Precision@K와 Recall@K 계산
NMF_model_precisions, NMF_model_recalls = precision_recall_at_k(NMF_model_predictions_list, k=5, threshold=3.5)

# Precision과 Recall 평균 계산
NMF_model_avg_precision = sum(prec for prec in NMF_model_precisions.values()) / len(NMF_model_precisions)
NMF_model_avg_recall = sum(rec for rec in NMF_model_recalls.values()) / len(NMF_model_recalls)

print("Average Precision@5(NMF_model):", NMF_model_avg_precision)
print("Average Recall@5(NMF_model):", NMF_model_avg_recall)

Average Precision@5(NMF_model): 0.7787733245004048
Average Recall@5(NMF_model): 0.3677391452138514
